In [1]:
import numpy as np
from scipy.optimize import linprog

# Coste por kg
costs = np.array([2.0, 3.0, 4.0, 1.5])

# Restricciones de desigualdad (Ax <= b)

# Pureza: >= 70 -> -pureza <= -70
A_pureza = [-80, -60, -90, -50]
b_pureza = -70

# Densidad inferior: >= 1.0 -> -densidad <= -1.0
A_dens_inf = [-1.2, -1.0, -0.8, -1.5]
b_dens_inf = -1.0

# Densidad superior: <= 1.3
A_dens_sup = [1.2, 1.0, 0.8, 1.5]
b_dens_sup = 1.3

# Toxicidad: <= 2
A_toxicidad = [1, 3, 0.5, 4]
b_toxicidad = 2

# Ingrediente A mínimo: x_A >= 0.20 -> -x_A <= -0.20
A_Amin = [-1, 0, 0, 0]
b_Amin = -0.20

# Ingrediente D máximo: x_D <= 0.25
A_Dmax = [0, 0, 0, 1]
b_Dmax = 0.25

A_ub = [A_pureza, A_dens_inf, A_dens_sup, A_toxicidad, A_Amin, A_Dmax]
b_ub = [b_pureza, b_dens_inf, b_dens_sup, b_toxicidad, b_Amin, b_Dmax]

# Restricción de igualdad: suma = 1
A_eq = [[1, 1, 1, 1]]
b_eq = [1]

# Bounds
bounds = [(0, 1) for _ in range(4)]

# Resolver
result = linprog(
    c=costs,
    A_ub=A_ub,
    b_ub=b_ub,
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=bounds,
    method='highs'
)

# Mostrar resultados
if result.success:
    x = result.x
    total_cost = result.fun
    print("Solución óptima encontrada:")
    print(f" - Proporción A: {x[0]:.2f}")
    print(f" - Proporción B: {x[1]:.2f}")
    print(f" - Proporción C: {x[2]:.2f}")
    print(f" - Proporción D: {x[3]:.2f}")
    print(f" - Coste mínimo: {total_cost:.2f} €/kg")

    pureza = 80*x[0] + 60*x[1] + 90*x[2] + 50*x[3]
    densidad = 1.2*x[0] + 1.0*x[1] + 0.8*x[2] + 1.5*x[3]
    toxicidad = 1*x[0] + 3*x[1] + 0.5*x[2] + 4*x[3]

    print(f" - Pureza final: {pureza:.2f}% (≥70%)")
    print(f" - Densidad final: {densidad:.2f} g/cm3 (1.0 - 1.3)")
    print(f" - Toxicidad final: {toxicidad:.2f}% (≤2%)")
else:
    print("No se encontró solución factible.")


Solución óptima encontrada:
 - Proporción A: 0.75
 - Proporción B: 0.00
 - Proporción C: 0.00
 - Proporción D: 0.25
 - Coste mínimo: 1.88 €/kg
 - Pureza final: 72.50% (≥70%)
 - Densidad final: 1.27 g/cm3 (1.0 - 1.3)
 - Toxicidad final: 1.75% (≤2%)


In [2]:
# Restricciones adicionales:
# Ingrediente máximo: x_i ≤ 0.60 para todos
A_max = [
    [1, 0, 0, 0],  # A ≤ 0.60
    [0, 1, 0, 0],  # B ≤ 0.60
    [0, 0, 1, 0],  # C ≤ 0.60
    [0, 0, 0, 1]   # D ≤ 0.60
]
b_max = [0.60, 0.60, 0.60, 0.60]

# Ingredientes B y C mínimos: x_B ≥ 0.10, x_C ≥ 0.10 -> -x ≤ -0.10
A_min = [
    [0, -1, 0, 0],  # B ≥ 0.10
    [0, 0, -1, 0]   # C ≥ 0.10
]
b_min = [-0.10, -0.10]

# Combinamos con restricciones anteriores
A_ub_new = A_ub + A_max + A_min
b_ub_new = b_ub + b_max + b_min

# Resolver nuevamente
result2 = linprog(
    c=costs,
    A_ub=A_ub_new,
    b_ub=b_ub_new,
    A_eq=A_eq,
    b_eq=b_eq,
    bounds=bounds,
    method='highs'
)

# Mostrar resultados del escenario técnico
if result2.success:
    x2 = result2.x
    total_cost2 = result2.fun
    print("Escenario avanzado: restricciones técnicas adicionales aplicadas")
    print(f" - Proporción A: {x2[0]:.2f}")
    print(f" - Proporción B: {x2[1]:.2f}")
    print(f" - Proporción C: {x2[2]:.2f}")
    print(f" - Proporción D: {x2[3]:.2f}")
    print(f" - Coste mínimo: {total_cost2:.2f} €/kg")

    pureza2 = 80*x2[0] + 60*x2[1] + 90*x2[2] + 50*x2[3]
    densidad2 = 1.2*x2[0] + 1.0*x2[1] + 0.8*x2[2] + 1.5*x2[3]
    toxicidad2 = 1*x2[0] + 3*x2[1] + 0.5*x2[2] + 4*x2[3]

    print(f" - Pureza final: {pureza2:.2f}% (≥70%)")
    print(f" - Densidad final: {densidad2:.2f} g/cm3 (1.0 - 1.3)")
    print(f" - Toxicidad final: {toxicidad2:.2f}% (≤2%)")
else:
    print("No se encontró solución factible para el escenario avanzado.")


Escenario avanzado: restricciones técnicas adicionales aplicadas
 - Proporción A: 0.55
 - Proporción B: 0.10
 - Proporción C: 0.10
 - Proporción D: 0.25
 - Coste mínimo: 2.18 €/kg
 - Pureza final: 71.50% (≥70%)
 - Densidad final: 1.22 g/cm3 (1.0 - 1.3)
 - Toxicidad final: 1.90% (≤2%)
